In [1]:
import requests
import json
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import date
from api_keys import yelp_key

In [2]:
params = {
        "location": "Fremont, CA",
        "radius": 16093,
        "categories": "restaurants, All",
        "sort_by": "review_count",
        "limit": 50
    }
headers = {
        "Authorization": "Bearer %s" % yelp_key,
    }
search_url = "https://api.yelp.com/v3/businesses/search"
searches = requests.get(search_url, headers=headers, params=params).json()

In [3]:
searches

{'businesses': [{'id': 'avlVbL84dBfLwBIFNJ3BpA',
   'alias': 'sen-dai-sushi-milpitas',
   'name': 'Sen Dai Sushi',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/UdOMSkNlIWUDcQQEAVjDJQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/sen-dai-sushi-milpitas?adjust_creative=xoRp6DMco5IR-CxSe_4F2A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=xoRp6DMco5IR-CxSe_4F2A',
   'review_count': 3254,
   'categories': [{'alias': 'japanese', 'title': 'Japanese'},
    {'alias': 'sushi', 'title': 'Sushi Bars'}],
   'rating': 4.0,
   'coordinates': {'latitude': 37.4329964646481, 'longitude': -121.909096},
   'transactions': ['pickup'],
   'price': '$$',
   'location': {'address1': '224 N Abel St',
    'address2': None,
    'address3': '',
    'city': 'Milpitas',
    'zip_code': '95035',
    'country': 'US',
    'state': 'CA',
    'display_address': ['224 N Abel St', 'Milpitas, CA 95035']},
   'phone': '+14082631472',
   'display_phone': '(408) 263-147

In [4]:
# create list of offset values to iterate through later
offset_list = list(np.arange(0, 200, 50))
offset_list

[0, 50, 100, 150]

In [5]:
# create function that will query yelp for restaurants with the most reviews, within 10 miles
# based on your location parameter

def yelp(location):
# params dictionary to update each iteration
    params = {
        "location": f"{location}",
        "radius": 16093,
        "categories": "restaurants, All",
        "sort_by": "review_count",
        "limit": 50
    }
    
# define header to authorize our query search
    headers = {
        "Authorization": "Bearer %s" % yelp_key,
    }
    
# create empty list to store our query results
    search_list = []
    
    search_url = "https://api.yelp.com/v3/businesses/search"
    
# we are limited to 50 results per query
# query up to 200 results with for loop that iterates through offset parameters
# and appends 50 results to our empty list each iteration
    for z in offset_list:
        params["offset"] = z
        searches = requests.get(search_url, headers=headers, params=params).json()
        print(f"Total of {z+50} restaurants processed...")
        search_list.append(searches)
    
# create 15 empty lists to store specific results from our json
# to store in a dataframe later
    id = []
    alias = []
    name = []
    is_closed = []
    review_count = []
    
    categories = []
    rating = []
    transactions = []
    price = []
    address = []
    
    city = []    
    country = []
    state = []
    display_address = []
    phone = []
        
# for loop to iterate through query results and append data to our lists
    for y in range(4):        
        for x in range(0, 50):
            id_search = search_list[y]["businesses"][x]["id"]
            id.append(id_search)
            alias_search = search_list[y]["businesses"][x]["alias"]
            alias.append(alias_search)
            name_search = search_list[y]["businesses"][x]["name"]
            name.append(name_search)
            is_closed_search = search_list[y]["businesses"][x]["is_closed"]
            is_closed.append(is_closed_search)
            review_count_search = search_list[y]["businesses"][x]["review_count"]
            review_count.append(review_count_search)
        
            categories_search = search_list[y]["businesses"][x]["categories"][0]["title"]
            categories.append(categories_search)
            rating_search = search_list[y]["businesses"][x]["rating"]
            rating.append(rating_search)
            transactions_search = search_list[y]["businesses"][x]["transactions"]
            transactions.append(transactions_search)
            price_search = search_list[y]["businesses"][x]["price"]
            price.append(price_search)
            address_search = search_list[y]["businesses"][x]["location"]["address1"]
            address.append(address_search)
        
            city_search = search_list[y]["businesses"][x]["location"]["city"]
            city.append(city_search)
            country_search = search_list[y]["businesses"][x]["location"]["country"]
            country.append(country_search)
            state_search = search_list[y]["businesses"][x]["location"]["state"]
            state.append(state_search)
            display_address_search = search_list[y]["businesses"][x]["location"]["display_address"]
            display_address.append(display_address_search)
            phone_search = search_list[y]["businesses"][x]["phone"]
            phone.append(phone_search)
        
# store all results in a new dataframe
    cat_df = pd.DataFrame({"id": id, "alias": alias,
                            "name": name, "is_closed": is_closed,
                            "review_count": review_count, "categories": categories,
                            "rating": rating, "transactions": transactions,
                            "price": price, "address": address, "city": city,
                            "country": country, "state": state, "display_address": display_address,
                            "phone": phone})
    
# sort new dataframe by highest rating, descending order
    cat_df = cat_df.sort_values("rating", ascending=False)
    
# have function return the dataframe created
    return cat_df

In [6]:
# run function with location string "Fremont, CA" and store results
fremont_ca = yelp("Fremont, CA")
fremont_ca

Total of 50 restaurants processed...
Total of 100 restaurants processed...
Total of 150 restaurants processed...
Total of 200 restaurants processed...


,id,alias,name,is_closed,review_count,categories,rating,transactions,price,address,city,country,state,display_address,phone
17,ud9ocsQHI7h3zNO7FdOFYQ,zareens-mountain-view-3,Zareen's,False,1804,Halal,4.5,"[pickup, delivery]",$$,1477 Plymouth St,Mountain View,US,CA,"[1477 Plymouth St, Ste C, Mountain View, CA 94...",+16506410335
53,_YDK0w05YAO99aQudHGpkA,philz-coffee-palo-alto-5,Philz Coffee,False,1153,Coffee & Tea,4.5,[delivery],$,3191 Middlefield Rd,Palo Alto,US,CA,"[3191 Middlefield Rd, Palo Alto, CA 94303]",+16502519798
181,OPuFiXxckMS6U5KZjQWYFw,dittmers-gourmet-meats-and-wurst-haus-los-altos-2,Dittmer's Gourmet Meats & Wurst-Haus,False,678,Meat Shops,4.5,"[pickup, delivery]",$$,4540 El Camino Real,Los Altos,US,CA,"[4540 El Camino Real, Los Altos, CA 94022]",+16509413800
85,HYbKSGCg8hqLw6J_X7qbyQ,acqua-e-farina-hayward,Acqua e Farina,False,974,Italian,4.5,[delivery],$$,22622 Main St,Hayward,US,CA,"[22622 Main St, Hayward, CA 94541]",+15108881568
42,tEnhREqfSe8MBwqtdcjL9A,dish-n-dash-milpitas-milpitas-3,Dish N Dash Milpitas,False,1246,Middle Eastern,4.5,"[pickup, delivery]",$$,181 Ranch Dr,Milpitas,US,CA,"[181 Ranch Dr, Milpitas, CA 95035]",+14082626949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,bWmk1Bx-Ad7NxGbKryeYjQ,anjappar-chettinad-indian-restaurant-milpitas,Anjappar Chettinad Indian Restaurant,False,932,Indian,2.5,"[pickup, delivery]",$$,458 Barber Ln,Milpitas,US,CA,"[458 Barber Ln, Milpitas, CA 95035]",+14084355500
169,LenRgAe-8kjyyM-ZGkL7Iw,saravanaa-bhavan-fremont-2,Saravanaa Bhavan,False,705,Indian,2.5,"[pickup, delivery]",$$,3720 Mowry Ave,Fremont,US,CA,"[3720 Mowry Ave, Fremont, CA 94538]",+15107917755
174,-UFiUQkwnVLNsut2lKG31g,world-gourmet-buffet-fremont,World Gourmet Buffet,False,694,Buffets,2.0,[],$$,6010 Stevenson Blvd,Fremont,US,CA,"[6010 Stevenson Blvd, Fremont, CA 94538]",+15104906888
113,pmdyX06bwGFrcBNfzmb8aQ,buffalo-wild-wings-union-city-22,Buffalo Wild Wings,False,850,Chicken Wings,2.0,[delivery],$$,32135 Union Landing Blvd,Union City,US,CA,"[32135 Union Landing Blvd, Union City, CA 94587]",+15104712329


In [7]:
# export dataframe of results with today's date
today = date.today()
d_today = today.strftime("%d_%m_%Y")

fremont_ca.to_csv(f"FremontCA_Restaurants_{d_today}.csv", index=False, header=True)

In [8]:
# examine unique values for particular columns
fremont_ca["city"].value_counts()

Fremont          50
Milpitas         39
Mountain View    34
Union City       15
Hayward          14
Newark           10
Pleasanton       10
San Jose          8
Sunnyvale         5
Palo Alto         5
Santa Clara       5
Los Altos         4
San Lorenzo       1
Name: city, dtype: int64